# Data Feature Engineering

In [ ]:
# %pip install pandas numpy scikit-learn xgboost pyarrow

## 0. 데이터 불러오기 (Reading Data)

In [ ]:
# import pandas as pd

# train_df = pd.read_csv("../data/all_train.csv")
# test_df = pd.read_csv("../data/all_test.csv")

In [ ]:
train_df.head()

In [ ]:
train_df.shape

In [ ]:
# # Segment 포함된 train_df를 기준으로 처리
# train_df = train_df.iloc[:, 1:]
# for col in train_df.columns:
#     if train_df[col].isnull().any():
#         train_df[col].fillna(0, inplace=True)


# # Feature / Label 분리
# y = train_df["Segment"]
# X = train_df.drop(columns=["Segment"]).select_dtypes(exclude=['object'])

# # Label 인코딩
# from sklearn.preprocessing import LabelEncoder
# label_encoder = LabelEncoder()
# y_encoded = label_encoder.fit_transform(y)

# # 스케일링
# from sklearn.preprocessing import StandardScaler
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X)

# # Train/Test 분할
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
# )


In [ ]:
# import time
# from sklearn.metrics import f1_score
# from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
# from sklearn.linear_model import LogisticRegression
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
# from sklearn.metrics import classification_report

# models = [
#     ("LogisticRegression", LogisticRegression(max_iter=1000)),
#     ("RandomForest (depth=5)", RandomForestClassifier(max_depth=5, n_estimators=100, random_state=42)),
#     ("RandomForest (depth=10)", RandomForestClassifier(max_depth=10, n_estimators=100, random_state=42)),
#     ("XGBoost (depth=3)", XGBClassifier(max_depth=3, use_label_encoder=False, eval_metric='mlogloss')),
#     ("XGBoost (depth=6)", XGBClassifier(max_depth=6, use_label_encoder=False, eval_metric='mlogloss')),
#     ("LGBM (depth=3)", LGBMClassifier(max_depth=3)),
#     ("LGBM (depth=6)", LGBMClassifier(max_depth=6)),
#     ("GradientBoosting", GradientBoostingClassifier(max_depth=4, n_estimators=100))
# ]

# # 결과 저장용 리스트
# results = []

# for name, model in models:
#     print(f"\n🔍 모델: {name}")
    
#     start_time = time.time()
#     model.fit(X_train, y_train)
#     y_pred = model.predict(X_test)
#     elapsed_time = time.time() - start_time
    
#     f1 = f1_score(y_test, y_pred, average='macro')
#     print(f"🧮 F1_macro score: {f1:.4f}")
    
#     # 저장
#     results.append({
#         "Model": name,
#         "F1_macro": f1,
#         "Time(s)": round(elapsed_time, 2)
#     })

#     # 상세 리포트도 원하면 출력 가능
#     print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# # 결과를 데이터프레임으로 정리
# results_df = pd.DataFrame(results)
# results_df = results_df.sort_values(by="F1_macro", ascending=False).reset_index(drop=True)

# # 보기 좋게 출력
# from IPython.display import display
# display(results_df)

## XGBoost Depth 비교를 통한 최적화

In [1]:
import pandas as pd

train_df = pd.read_csv("../data/all_train.csv")
# test_df = pd.read_csv("../data/all_test.csv")

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17696\427125886.py:3: DtypeWarning: Columns (300,385) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv("../data/all_train.csv")


In [ ]:
# Segment 포함된 train_df를 기준으로 처리
train_df = train_df.iloc[:, 1:]
for col in train_df.columns:
    if train_df[col].isnull().any():
        train_df[col].fillna(0, inplace=True)


# Feature / Label 분리
y = train_df["Segment"]
X = train_df.drop(columns=["Segment"]).select_dtypes(exclude=['object'])

# Label 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# 수치형 Feature와 Label을 포함한 데이터 저장
numeric_df = pd.concat([pd.DataFrame(X, columns=train_df.drop(columns=["Segment"]).select_dtypes(exclude=['object']).columns), train_df["Segment"].reset_index(drop=True)], axis=1)
numeric_df.to_csv("../data/numeric_train.csv", index=False)


# Train/Test 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=42
)

C:\Users\SSAFY\AppData\Local\Temp\ipykernel_17696\305744020.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  train_df[col].fillna(0, inplace=True)


## Numeric 데이터만 따로 불러와서 계산

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

numeric_df = pd.read_csv("../data/numeric_train.csv")

# Feature / Label 분리
y = numeric_df["Segment"]
X = numeric_df.drop(columns=["Segment"])

# Label 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=8432)

In [3]:
numeric_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 809 entries, 기준년월 to Segment
dtypes: float64(61), int64(747), object(1)
memory usage: 14.5+ GB


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Columns: 808 entries, 기준년월 to 혜택수혜율_B0M
dtypes: float64(61), int64(747)
memory usage: 14.4 GB


###  depth만 변화를 주면서 계산

In [2]:
import time
import joblib
import gc
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report

depth_values = [6, 7, 8, 9, 10, 11, 12, 13]

for depth in depth_values:
    print(f"\n🌲 XGBoost max_depth={depth}") 
    model = XGBClassifier(max_depth=depth, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    
    start_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed_time = time.time() - start_time

    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"🔍 F1_macro score: {f1:.4f}")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))
    print(f"⏱ 학습 및 예측 시간: {elapsed_time:.2f}초")

    # 모델 저장 (파일명에 max_depth와 f1 score 포함)
    model_filename = f"xgboost_depth{depth}_f1{f1:.4f}.joblib"
    joblib.dump(model, model_filename)
    print(f"💾 모델 저장 완료: {model_filename}")

    # 메모리 해제를 위해 모델 변수 삭제 및 가비지 컬렉션 수행
    del model
    gc.collect()



🌲 XGBoost max_depth=6


c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [09:55:05] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.7870
              precision    recall  f1-score   support

           A       0.95      0.62      0.75       194
           B       1.00      0.59      0.74        29
           C       0.78      0.69      0.73     25518
           D       0.77      0.72      0.75     69848
           E       0.96      0.97      0.96    384411

    accuracy                           0.92    480000
   macro avg       0.89      0.72      0.79    480000
weighted avg       0.92      0.92      0.92    480000

⏱ 학습 및 예측 시간: 353.59초
💾 모델 저장 완료: xgboost_depth6_f10.7870.joblib

🌲 XGBoost max_depth=7


c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [10:00:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.8020
              precision    recall  f1-score   support

           A       0.98      0.65      0.78       194
           B       1.00      0.59      0.74        29
           C       0.80      0.72      0.76     25518
           D       0.79      0.75      0.77     69848
           E       0.96      0.98      0.97    384411

    accuracy                           0.93    480000
   macro avg       0.91      0.73      0.80    480000
weighted avg       0.93      0.93      0.93    480000

⏱ 학습 및 예측 시간: 314.54초
💾 모델 저장 완료: xgboost_depth7_f10.8020.joblib

🌲 XGBoost max_depth=8


...

🔍 F1_macro score: 0.8197
              precision    recall  f1-score   support

           A       0.97      0.64      0.77       194
           B       1.00      0.48      0.65        29
           C       0.89      0.81      0.85     25518
           D       0.87      0.84      0.85     69848
           E       0.97      0.98      0.98    384411

    accuracy                           0.95    480000
   macro avg       0.94      0.75      0.82    480000
weighted avg       0.95      0.95      0.95    480000

⏱ 학습 및 예측 시간: 446.49초
💾 모델 저장 완료: xgboost_depth11_f10.8197.joblib

🌲 XGBoost max_depth=12


c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [10:31:13] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.8094
              precision    recall  f1-score   support

           A       0.97      0.63      0.76       194
           B       1.00      0.41      0.59        29
           C       0.90      0.81      0.86     25518
           D       0.88      0.85      0.86     69848
           E       0.97      0.99      0.98    384411

    accuracy                           0.96    480000
   macro avg       0.95      0.74      0.81    480000
weighted avg       0.96      0.96      0.96    480000

⏱ 학습 및 예측 시간: 503.62초
💾 모델 저장 완료: xgboost_depth12_f10.8094.joblib

🌲 XGBoost max_depth=13


c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [10:39:39] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.8206
              precision    recall  f1-score   support

           A       0.94      0.61      0.74       194
           B       1.00      0.48      0.65        29
           C       0.91      0.81      0.86     25518
           D       0.89      0.85      0.87     69848
           E       0.97      0.99      0.98    384411

    accuracy                           0.96    480000
   macro avg       0.94      0.75      0.82    480000
weighted avg       0.96      0.96      0.96    480000

⏱ 학습 및 예측 시간: 582.62초
💾 모델 저장 완료: xgboost_depth13_f10.8206.joblib


In [ ]:
import time
import joblib
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import f1_score, classification_report
from IPython.display import display

depth_values = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16]
xgb_results = []    
best_model = None
best_f1 = 0
best_depth = None

for depth in depth_values:
    print(f"\n🌲 XGBoost max_depth={depth}") 
    model = XGBClassifier(max_depth=depth, use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    
    start_time = time.time()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    elapsed_time = time.time() - start_time

    f1 = f1_score(y_test, y_pred, average='macro')
    print(f"🔍 F1_macro score: {f1:.4f}")
    print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))

    xgb_results.append({
        "max_depth": depth,
        "F1_macro": f1,
        "Time(s)": round(elapsed_time, 2)
    })

    # 가장 성능 좋은 모델 저장
    if f1 > best_f1:
        best_f1 = f1
        best_model = model
        best_depth = depth

# 결과 정리
xgb_results_df = pd.DataFrame(xgb_results).sort_values(by="F1_macro", ascending=False).reset_index(drop=True)
display(xgb_results_df)

# 모델 저장
model_filename = f"best_xgboost_depth{best_depth}_f1{best_f1:.4f}.joblib"
joblib.dump(best_model, model_filename)
print(f"\n💾 Best XGBoost 모델 저장 완료: {model_filename}")



🌲 XGBoost max_depth=6


c:\Users\SSAFY\Desktop\개인폴더\DACON\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:36:03] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.7821
              precision    recall  f1-score   support

           A       0.98      0.65      0.78       194
           B       1.00      0.52      0.68        29
           C       0.78      0.69      0.74     25518
           D       0.77      0.72      0.75     69848
           E       0.96      0.97      0.96    384411

    accuracy                           0.92    480000
   macro avg       0.90      0.71      0.78    480000
weighted avg       0.92      0.92      0.92    480000


🌲 XGBoost max_depth=7


c:\Users\SSAFY\Desktop\개인폴더\DACON\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:41:23] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.8063
              precision    recall  f1-score   support

           A       0.97      0.68      0.80       194
           B       1.00      0.59      0.74        29
           C       0.80      0.72      0.76     25518
           D       0.79      0.75      0.77     69848
           E       0.96      0.98      0.97    384411

    accuracy                           0.93    480000
   macro avg       0.90      0.74      0.81    480000
weighted avg       0.93      0.93      0.93    480000


🌲 XGBoost max_depth=8


c:\Users\SSAFY\Desktop\개인폴더\DACON\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:46:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.7909
              precision    recall  f1-score   support

           A       0.98      0.66      0.79       194
           B       1.00      0.45      0.62        29
           C       0.83      0.75      0.78     25518
           D       0.81      0.77      0.79     69848
           E       0.96      0.98      0.97    384411

    accuracy                           0.94    480000
   macro avg       0.92      0.72      0.79    480000
weighted avg       0.93      0.94      0.93    480000


🌲 XGBoost max_depth=9


c:\Users\SSAFY\Desktop\개인폴더\DACON\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:52:32] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


🔍 F1_macro score: 0.8137
              precision    recall  f1-score   support

           A       0.98      0.66      0.79       194
           B       1.00      0.52      0.68        29
           C       0.85      0.77      0.81     25518
           D       0.83      0.79      0.81     69848
           E       0.97      0.98      0.97    384411

    accuracy                           0.94    480000
   macro avg       0.93      0.75      0.81    480000
weighted avg       0.94      0.94      0.94    480000


🌲 XGBoost max_depth=10


c:\Users\SSAFY\Desktop\개인폴더\DACON\.venv\lib\site-packages\xgboost\core.py:158: UserWarning: [17:58:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


In [4]:
import pandas as pd

# 모든 행과 열, 그리고 각 셀의 내용을 생략 없이 출력하도록 설정
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

numeric_df = pd.read_csv("../data/numeric_train.csv")



,기준년월,남녀구분코드,회원여부_이용가능,회원여부_이용가능_CA,회원여부_이용가능_카드론,소지여부_신용,소지카드수_유효_신용,소지카드수_이용가능_신용,입회일자_신용,입회경과개월수_신용,...,변동률_할부평잔,변동률_CA평잔,변동률_RVCA평잔,변동률_카드론평잔,변동률_잔액_B1M,변동률_잔액_일시불_B1M,변동률_잔액_CA_B1M,혜택수혜율_R3M,혜택수혜율_B0M,Segment
0,201807,2,1,1,0,1,1,1,20130101,67,...,1.042805,0.999700,0.999998,0.999998,0.261886,0.270752,0.000000,1.044401,1.280543,D
1,201807,1,1,1,1,1,1,1,20170801,12,...,0.905663,0.999998,0.999998,0.999998,-0.563388,-0.670348,0.000000,0.000000,0.000000,E
2,201807,1,1,1,0,1,1,1,20080401,124,...,1.993590,0.852567,0.999998,0.999998,-0.046516,0.058114,-0.014191,0.524159,1.208420,C
3,201807,2,1,1,0,1,2,2,20160501,27,...,1.050646,0.999877,0.999998,0.999998,0.023821,0.258943,0.000000,0.880925,1.657124,D
4,201807,2,1,1,1,1,1,1,20180601,2,...,0.999998,0.999998,0.999998,0.999998,0.000000,0.000000,0.000000,0.000000,0.000000,E


In [6]:
# Feature / Label 분리
y = numeric_df["Segment"]
X = numeric_df.drop(columns=["Segment"])

# Label 인코딩
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_encoded, test_size=0.2, stratify=y_encoded, random_state=8432)

In [ ]:
# %pip install optuna

You should consider upgrading via the 'c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\Scripts\python.exe -m pip install --upgrade pip' command.



     -------------------------------------- 383.6/383.6 KB 8.1 MB/s eta 0:00:00
  Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
  Using cached PyYAML-6.0.2-cp39-cp39-win_amd64.whl (162 kB)
     ------------------------------------- 231.8/231.8 KB 13.8 MB/s eta 0:00:00
     ---------------------------------------- 2.1/2.1 MB 19.3 MB/s eta 0:00:00
     ---------------------------------------- 78.5/78.5 KB ? eta 0:00:00
  Using cached greenlet-3.1.1-cp39-cp39-win_amd64.whl (298 kB)
  Using cached MarkupSafe-3.0.2-cp39-cp39-win_amd64.whl (15 kB)


In [9]:
import optuna
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

# Optuna 목적 함수 정의
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 500),
        'max_depth': trial.suggest_int('max_depth', 6, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'gamma': trial.suggest_float('gamma', 0, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'use_label_encoder': False,
        'eval_metric': 'logloss'
    }

    model = XGBClassifier(**params)
    score = cross_val_score(model, X_train, y_train, cv=5, scoring='accuracy').mean()
    return score

# Optuna 튜닝 수행
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 최적 파라미터 확인
print("Best trial:")
print(f"  Value: {study.best_trial.value}")
print("  Params:")
for key, value in study.best_trial.params.items():
    print(f"    {key}: {value}")

# 최적 파라미터로 모델 훈련 및 평가
best_params = study.best_trial.params
best_params['use_label_encoder'] = False
best_params['eval_metric'] = 'logloss'

model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

# 예측 및 성능 평가
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"Test Accuracy: {acc:.4f}")

c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-03-28 09:10:07,554] A new study created in memory with name: no-name-4f94b1a1-78a6-4981-a917-6086f169615f
c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [09:12:06] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py:158: UserWarning: [09:29:36] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgb

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
1 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\sklearn\model_selection\_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\sklearn.py", line 1599, in fit
    self._Booster = train(
  File "c:\Users\SSAFY\Desktop\개인폴더\DACON\01_신용카드_고객_세그먼트_분류_AI_경진대회\venv\lib\site-packages\xgboost\core.py", line 726, in 

## 📌 1. 결측치 처리 (Missing Values Handling)
- 평균, 중앙값, 최빈값으로 대체
- 특정 값으로 대체 (예: -1, 0, 기타 의미 있는 값)
- 보간(interpolation)을 통한 누락값 채우기
- KNN 또는 머신러닝 기반 추정을 이용한 채우기
- 결측치가 많은 피처 제거

## 📌 2. 이상치 처리 (Outlier Treatment)
- 통계적 방법 (IQR, Z-score)으로 이상치 탐지 및 제거
- 윈저라이징(Winsorizing): 극단값을 최대 또는 최소값으로 제한
- 이상치를 별도 피처로 생성하여 표현 (이진 플래그 형태)

## 📌 3. 인코딩 (Encoding Categorical Variables)
- 원-핫 인코딩(One-Hot Encoding)
- 라벨 인코딩(Label Encoding)
- 빈도 기반 인코딩(Frequency Encoding)
- 순서 인코딩(Ordinal Encoding)
- 타깃 인코딩(Target Encoding)

## 📌 4. 피처 스케일링 및 정규화 (Scaling and Normalization)
- 최소-최대 정규화(Min-Max Scaling)
- 표준화(Standardization, Z-score Scaling)
- 로그(log), 박스-콕스(Box-Cox), 여(Yeo)-존슨 변환 등
- 로버스트 스케일링(Robust Scaling, 이상치 존재 시)

## 📌 5. 피처 생성 (Feature Creation)
- 산술 연산(덧셈, 뺄셈, 곱셈, 나눗셈)을 통한 새로운 피처 생성
- 다항식 특성(Polynomial Features)
- 도메인 기반 파생변수 생성 (예: BMI = 몸무게/키²)
- 날짜/시간 기반 파생변수 (요일, 주말 여부, 계절 등)
- 상호작용(interaction) 특성 생성

## 📌 6. 피처 추출 (Feature Extraction)
- PCA (Principal Component Analysis, 주성분 분석)
- SVD (Singular Value Decomposition, 특이값 분해)
- t-SNE, UMAP 등 차원 축소 기법
- 클러스터링 기반 피처 생성 (예: 군집 ID)
- AutoEncoder를 통한 특성 추출 (딥러닝 방식)

## 📌 7. 피처 선택 (Feature Selection)
- 상관관계 분석을 통한 다중공선성 제거
- 피처 중요도 기반 선택 (랜덤포레스트, XGBoost, LightGBM 등)
- 재귀적 피처 제거 (Recursive Feature Elimination, RFE)
- 통계적 테스트 기반 피처 선택 (ANOVA, Chi-square test 등)
- Wrapper 방법 (모델 기반 탐색적 피처 선택)

## 📌 8. 불균형 데이터 처리 (Handling Imbalanced Data)
- 오버샘플링(Oversampling): SMOTE 및 변형(SMOTE-Tomek 등)
- 언더샘플링(Undersampling): Random, Tomek Links, Edited Nearest Neighbor
- 클래스 가중치(Class Weight) 적용
- 데이터 증강(Data Augmentation)

## 📌 9. 시계열 데이터 처리 (Time-Series Data Processing)
- 이동 평균(Moving average), 지수평활(Exponential smoothing) 등의 시계열 피처 생성
- 시간차(lag) 변수 생성
- 롤링 윈도우(rolling window)를 이용한 특성 추출
- 주기성 및 계절성 패턴 분석

## 📌 10. 지리적 데이터 처리 (Geospatial Data Processing)
- 좌표 변환(위도, 경도 → 거리, 방향 등)
- Geohash 또는 지도 기반 클러스터링
- 지리적 인접성을 이용한 피처 생성